# Setup

In [ ]:
import os
import pandas as pd

In [3]:
cwd = 'C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/Progetti/MAURICE/MAURICE_progetto/Dati'

In [ ]:
#carica dati arpa 2000-2023
metaarpa = pd.read_csv("C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/Dati/ARPA Lombardia/piezo/download_database_170924/meta_ARPA_Lombardia_2000_2023_merged.csv")
dbarpa = pd.read_csv("C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/Dati/ARPA Lombardia/piezo/download_database_170924/df_ARPA_Lombardia_2000_2023_merged.csv")
# seleziona piezo arpa necessari
codes = pd.read_csv('c:\\repos\\MAURICE\\data\\codes_arpa_maurice.csv')
ts_arpa = dbarpa.loc[dbarpa.CODICE.isin(codes.CODICE),:].copy()
ts_arpa.Data = pd.to_datetime(ts_arpa.Data)
# ottieni dati mensili
ts_arpa = ts_arpa.pivot_table(index = 'Data', columns='CODICE', values='head')
ts_arpa = ts_arpa.resample('1MS').mean()
#carica dati MAURICE 2023-2024
meta = pd.read_csv(os.path.join(cwd, 'Dati_piezometrici', 'struttura-webgis', '1-anagrafica_MAURICE.csv'))
ts = pd.read_csv(os.path.join(cwd, 'Dati_piezometrici', 'struttura-webgis', 'altro', '7-dati_piezo_MAURICE_csv.csv'))
#rimuovi da dati MAURICE i punti arpa
ts.drop(index = ts.loc[ts.codice_campagna == 'ARPA'].index, inplace=True)
ts.data = pd.to_datetime(ts.data)
ts = ts.loc[:, ['data', 'id_punto', 'quota_falda']]
#tieni metadati per i punti che hanno osservazioni
meta = meta.loc[meta.id_punto.isin(ts.id_punto.unique()), :]
#pivot per ottenere osservazioni sulle righe e punti sulle colonne
ts = ts.pivot_table(index = 'data', columns = 'id_punto', values = 'quota_falda')
# ottieni dati mensili
tsm = ts.resample('1MS').mean()
# unisci i due database
#meta
meta1 = metaarpa.loc[:, ['CODICE','X_WGS84', 'Y_WGS84']].copy()
meta2 = meta.loc[:, ['id_punto', 'x_sr2', 'y_sr2']].copy()

meta1.columns = ['id_punto', 'x_sr2', 'y_sr2']

metau = pd.concat([meta1, meta2], axis = 0)
metau.reset_index(inplace=True, drop=True)

#ts
tsu = tsm.merge(ts_arpa, how = 'outer', left_index=True, right_index=True)
tool = tsu.reset_index().melt(id_vars = ['index']).dropna()
tool.columns = ['date', 'id_punto', 'value']
st_df = tool.merge(metau, how = 'left', on = 'id_punto')
st_df.columns = ['date', 'id_punto', 'value', 'x', 'y']
rimuovi = [
    '410009', '150100132', '150100081', '150100005', 'PZ1B', 'LagoCapanna',
    '150120160', '150120134', '150120135', '150120149'
]
st_df = st_df.loc[~st_df.id_punto.isin(rimuovi),:]
# st_df.to_csv('data/st_kriging_data_points.csv')

## Parameters

In [35]:
coord = [
    484234.65167973784, #xmin
    5019178.630486382,  #ymin
    517495.35663193866, #xmax
    5055708.644651484   #ymax
]